# importing of required libraries

In [ ]:
#importing of required libraries
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import pandas as pd
import time

## writing a function to fecth all posts based on hashtags

In [ ]:

def give_credits(user,key,hashtags,csv_filepath):
    
    #initilisation of chrome
    driver = webdriver.Chrome(r"C:\Users\srikanth\Downloads\chromedriver.exe")
    #opening a linkedin log in page
    driver.get("https://www.linkedin.com/uas/login?session_redirect=%2Fvoyager%2FloginRedirect%2Ehtml&amp;fromSignIn=true&amp;trk=uno-reg-join-sign-in")
    
    #entring of log in credentials and clicking signin
    username = driver.find_element_by_id("username")
    password = driver.find_element_by_id("password")

    username.send_keys(user)
    password.send_keys(key)

    signin= driver.find_element_by_class_name("login__form_action_container ")
    signin.click()
    driver.get('https://www.linkedin.com/feed/')
    driver.maximize_window() # to maximize window
    
    # creating a customized link for opening bases on hasgtag words
    def hashtag_words( keys=[],*args):
        tags=[]
        temp=''
        for i in range(len(keys)):
            if i==0:
                word= "%23"+str(keys[i])
                tags.append(word)
            if i>0:
                word='%20%23'+str(keys[i])
                tags.append(word)
        k=temp.join(tags)
        final_link='https://www.linkedin.com/search/results/content/?keywords='+k+'&origin=GLOBAL_SEARCH_HEADER'
        return final_link
    
    #opening a page contains all given hashtag words
    driver.get(hashtag_words(keys=hashtags))
    
    # writing a infinte loop to scroll down
    SCROLL_PAUSE_TIME = 1.5

    # Get scroll height
    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait to load page
        time.sleep(SCROLL_PAUSE_TIME)

        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height
            
    # command to fetch all the information contaains in the page       
    content = driver.page_source
    soup = BeautifulSoup(content)    
    
    # fetching post,name,linkedin_member_link who posted 
    link_posts=soup.find_all('li',class_='search-content__result search-entity ember-view')
    print(len(link_posts))
    post=soup.find_all('div',class_='feed-shared-text__text-view feed-shared-text-view white-space-pre-wrap break-words ember-view')
    print(len(post))
    name=soup.find_all('span',class_='feed-shared-actor__name t-14 t-black t-bold hoverable-link-text')
    print(len(name))
    ref_link=soup.findAll('a',href=True, attrs={'class':'feed-shared-actor__container-link display-flex flex-grow-1 app-aware-link ember-view'})
    linkedin_post=[]
    linkedin_post_link=[]
    linkedin_name=[]
    linkedin_link=[]
    for i in range(len(post)):
        n=name[i].text
        print(n)
        linkedin_post.append(post[i].text)
        linkedin_name.append(name[i].text)
        linkedin_link.append(ref_link[i].get('href'))
        k='https://www.linkedin.com/feed/update/'+ link_posts[i].find('div').get('data-id') + '/'
        print(k)
        linkedin_post_link.append(k)
    
        linkedin_post= [s.strip() for s in linkedin_post]  
        linkedin_name= [s.strip() for s in linkedin_name] 
        linkedin_link= [s.strip() for s in linkedin_link] 
        linkedin_post_link= [s.strip() for s in linkedin_post_link] 


        linkedin_post= [s.replace('\n','') for s in linkedin_post]  
        linkedin_name= [s.replace('\n','') for s in linkedin_name] 
        linkedin_link= [s.replace('\n','') for s in linkedin_link]
        linkedin_post_link= [s.replace('\n','') for s in linkedin_post_link]

    #creating a data frame contains name,link and post details
    main_data={'Name':linkedin_name,
                'ref_links':linkedin_link,
                'post':linkedin_post,
                'linkedin_post_link': linkedin_post_link}
        
    post_df=pd.DataFrame(main_data)
    
    # created post_dataframe is saved to csv file 
    post_df.to_csv(csv_filepath)
        
    return post_df